In [2]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import tensorflow_model_optimization as tfmot
import ast
from collections import OrderedDict
from typing import Tuple

(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.fashion_mnist.load_data()
train_images = train_images / 255.0
test_images = test_images / 255.0

In [3]:
LOAD_PATH_Q_AWARE = "./model/" + "model_qware_final_01"

q_aware_model : tf.keras.Model
with tfmot.quantization.keras.quantize_scope():
    q_aware_model = tf.keras.models.load_model(LOAD_PATH_Q_AWARE)


def bit_flipper(value : int, bit_pos : int) -> int:
    """ Random bit flipper 
    -
    Obtains a value and bit position and flips it.
    - All values are in 8 bits, MSB have higher probability of getting flipped
    - It is assumed value is a signed 8 bit number """
    # Negative 2 Complement conversion
    if value < 0:
        value = (-value ^ 0xFF) + 1
    flip_mask = 1 << bit_pos
    flipped_value = value ^ flip_mask
    # Negative back conversion 2 Complement
    if flipped_value >= 128:
        flipped_value = -((flipped_value ^ 0xFF) + 1)
    return flipped_value

In [4]:
q_aware_test_loss, q_aware_test_acc = q_aware_model.evaluate(test_images, test_labels)
print('Test accuracy : ', "{:0.2%}".format(q_aware_test_acc))

313/313 [==============================] - 2s 6ms/step - loss: 0.2891 - accuracy: 0.9115
Test accuracy :  91.15%


In [7]:
bit_width = 8
quantized_and_dequantized = OrderedDict()
quantized = OrderedDict()
layer_index_list = []
keys_list = []

layer : tfmot.quantization.keras.QuantizeWrapperV2
for i, layer in enumerate(q_aware_model.layers):
    quantizer : tfmot.quantization.keras.quantizers.Quantizer
    weight : tf.Variable
    if hasattr(layer, '_weight_vars'):
        for weight, quantizer, quantizer_vars in layer._weight_vars:
            min_var = quantizer_vars['min_var']
            max_var = quantizer_vars['max_var']

            key = weight.name[:-2]
            layer_index_list.append(i)
            keys_list.append(key)
            quantized_and_dequantized[key] = quantizer(inputs = weight, training = False, weights = quantizer_vars)
            quantized[key] = np.round(quantized_and_dequantized[key] / max_var * (2**(bit_width-1)-1))

for key in quantized:
    # print("Fake Quantized")
    print(key, quantized[key].shape)
    if "dense" not in key:
        # print(quantized_and_dequantized[key][:,:,0,0])
        print(quantized[key][:,:,0,0])
    else:
        # print(quantized_and_dequantized[key][:,0])
        print(quantized[key][:,0])

conv2d/kernel (5, 5, 1, 32)
[[ -34.    9.   41.  -66. -127.]
 [ -70.    8.   57.   27.  -73.]
 [   5.   39.  -30.  -17.   33.]
 [  47.  -23.   -1.  -23.   -1.]
 [  39.   16.  -19.    4.   12.]]
conv2d_1/kernel (5, 5, 32, 64)
[[-18. -40. -32.   4.  -3.]
 [  5. -28. -42.  12.  20.]
 [ 47. -42. -14. -11.  32.]
 [ 41. -15.   0. -15.  17.]
 [ 35. -10.  -4.  17.  25.]]
conv2d_2/kernel (3, 3, 64, 96)
[[ 69.  51.  -2.]
 [  2. -98. -63.]
 [ 67.   5. -44.]]
dense_last/kernel (96, 10)
[  4.   1.  -6.   6.  -1. -10. -18.  20. -18.  -8.  -1.  -6. -30. -13.
  -8.  11.  13.   5. -35. -12. -24. -18.   8.   8.  -4. -27.  18.  15.
 -46. -23.  21. -42.  19.  12.  12.  -1.   9.  -8.   1.  18.  -6.  13.
  -2.  13.   8. -10.  12.  27.   5.  -1.  -5.  -5. -11.  13. -29. -25.
  -8.   4.  19.   4. -19. -26.  32.  15. -35. -12.   0.  31.  -8. -25.
  10.  21.  14.  14.  -9.   7. -24. -23. -15.  18.   6.  27.  26.  13.
   9.  34.   6. -25. -23. -14. -42. -17.  -2.   7. -18.   9.]


In [5]:
conv_idx = 0 
key = keys_list[conv_idx]
print(type(quantized[key][:,:,0,29]))
print(quantized[key][:,:,0,29])

<class 'numpy.ndarray'>
[[  22.    1.   73.  102.   -7.]
 [  56.   50.   37.   77.   68.]
 [ -75.   21.   -5.   69.   19.]
 [ -38.  -30.  -74.   52.   -7.]
 [ -66.  -65. -127.   23.   23.]]


In [9]:
df = pd.read_csv('Performance.csv')
position_disrupted_list = []
bit_flipped_list = []
for d in df['position_disrupted']:
    try:
        position_disrupted_list.append(ast.literal_eval(d))
    except:
        position_disrupted_list.append(tuple())
for d in df['bit_disrupted']:
    try:
        bit_flipped_list.append(int(d))
    except:
        bit_flipped_list.append(-1)

BIT_WIDTH = 8
out_list = []
conv_idx = 0 
key = keys_list[conv_idx]
layer_index = layer_index_list[conv_idx]
T_VARIABLES_KERNEL_INDEX = 0

pos = 2013
print(position_disrupted_list[pos])
print(bit_flipped_list[pos])
print(df['quantized_value'][pos])
print(quantized[key][position_disrupted_list[pos]])
for i, position in enumerate(position_disrupted_list):
    entry = {}
    entry['data'] = position
    if len(position) > 1:
        m_vars = {variable.name: variable for i, variable in enumerate(q_aware_model.layers[layer_index].non_trainable_variables) if keys_list[conv_idx] in variable.name}
        min_key = list(key for key in m_vars if "min" in key)[0]
        max_key = list(key for key in m_vars if "max" in key)[0]
        min_var = m_vars[min_key]
        max_var = m_vars[max_key]
        entry['min_var'] = min_var[position[-1]].numpy()
        entry['max_var'] = max_var[position[-1]].numpy()
        entry['original_weight_value'] = q_aware_model.layers[layer_index].trainable_variables[T_VARIABLES_KERNEL_INDEX][position].numpy()
        entry['quantized_value'] = quantized[key][position]
        entry['flipped_quantized_value'] = bit_flipper(int(quantized[key][position]), bit_flipped_list[i])
        entry['flipped_weight_value'] = entry['flipped_quantized_value'] * max_var.numpy()[position[-1]] / (2**(BIT_WIDTH - 1) - 1)
        # Laplacian calculation
        kernel = q_aware_model.layers[layer_index].trainable_variables[T_VARIABLES_KERNEL_INDEX][:,:,position[2],position[-1]].numpy() # Passed as copy
        original_laplacian = sp.ndimage.laplace(kernel)
        kernel[position[:2]] = entry['flipped_weight_value']
        new_laplacian = sp.ndimage.laplace(kernel)
        entry['original_laplacian'] = original_laplacian[position[:2]]
        entry['modified_laplacian'] = new_laplacian[position[:2]]
        # When assigning a whole array it is passed as reference
        # Important to avoid modifying the original values
        int_kernel = np.copy(quantized[key][:,:,position[2],position[-1]])
        original_int_laplacian = sp.ndimage.laplace(int_kernel)
        int_kernel[position[:2]] = entry['flipped_quantized_value']
        new_int_laplacian = sp.ndimage.laplace(int_kernel)    
        entry['original_int_laplacian'] = original_int_laplacian[position[:2]]
        entry['modified_int_laplacian'] = new_int_laplacian[position[:2]]
    else:
        entry['min_var'] = None
        entry['max_var'] = None
        entry['original_weight_value'] = None
        entry['quantized_value'] = None
        entry['flipped_quantized_value'] = None
        entry['flipped_weight_value'] = None
        entry['original_laplacian'] = None
        entry['modified_laplacian'] = None
        entry['original_int_laplacian'] = None
        entry['modified_int_laplacian'] = None
    out_list.append(entry)
out_df = pd.DataFrame(out_list)
out_df.to_csv('Max_Min.csv')

(3, 4, 0, 28)
6
-12.0
-12.0
